In [1]:
"""
This notebook implements an end-to-end ETL pipeline that:
1. Reads streaming data from Kafka
2. Transforms the data with Spark
3. Writes results to PostgreSQL (for analytics) and MinIO (for archival)
"""

'\nThis notebook implements an end-to-end ETL pipeline that:\n1. Reads streaming data from Kafka\n2. Transforms the data with Spark\n3. Writes results to PostgreSQL (for analytics) and MinIO (for archival)\n'

In [16]:
# Import libraries
import psycopg2
import sys
import time
import pandas as pd 
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import (
    col, from_json, current_timestamp, to_date, hour, dayofweek, when, lit, udf
)
from pyspark.sql.types import (
    StructType, StructField, StringType, TimestampType, 
    DoubleType, IntegerType, BooleanType
)
from functools import lru_cache
from typing import Any
from time import sleep 
from psycopg2 import sql
from psycopg2.extras import execute_batch
from psycopg2 import pool

In [17]:
# Create Spark session with MinIO/S3 support
spark = (SparkSession.builder
    .appName("SmartMeterETL")
    
    # JAR Configuration - Add Hadoop AWS and related jars
    .config("spark.jars", ",".join([
        "/opt/spark/jars/spark-sql-kafka-0-10_2.12-3.5.0.jar",
        "/opt/spark/jars/kafka-clients-3.5.0.jar",
        "/opt/spark/jars/kafka_2.12-3.5.0.jar",
        "/opt/spark/jars/commons-pool2-2.11.1.jar",
        "/opt/spark/jars/lz4-java-1.8.0.jar",
        "/opt/spark/jars/snappy-java-1.1.10.1.jar",
        "/opt/spark/jars/hadoop-aws-3.3.4.jar",
        "/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar"
    ]))
    
    # Classpath Configuration
    .config("spark.driver.extraClassPath", "/opt/spark/jars/*")
    .config("spark.executor.extraClassPath", "/opt/spark/jars/*")
    .config("spark.executor.userClassPathFirst", "true")
    
    # MinIO/S3 Configuration
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9002")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

    # MinIO/S3 Optimizations:
    .config("spark.hadoop.fs.s3a.fast.upload.buffer", "disk")
    .config("spark.hadoop.fs.s3a.fast.upload.active.blocks", "4")
    .config("spark.hadoop.fs.s3a.multipart.threshold", "128M")
    .config("spark.hadoop.fs.s3a.fast.upload", "true")
    .config("spark.hadoop.fs.s3a.connection.timeout", "100000")

    # Prevent temp files from appearing in MinIO/S3
    .config("spark.hadoop.fs.s3a.committer.name", "directory")
    .config("spark.hadoop.mapreduce.outputcommitter.factory.scheme.s3a", 
            "org.apache.hadoop.fs.s3a.commit.S3ACommitterFactory")

    # Kafka Specific Settings
    .config("spark.sql.streaming.kafka.useDeprecatedOffsetFetching", "false")
    .config("spark.kafka.consumer.cache.enabled", "false")
    .config("spark.streaming.backpressure.enabled", "true")
    .config("spark.streaming.kafka.maxRatePerPartition", "1000")
    
    # JVM Options
    .config("spark.driver.extraJavaOptions",
           "-Dio.netty.tryReflectionSetAccessible=true " +
           "--add-opens=java.base/sun.nio.ch=ALL-UNNAMED " +
           "--add-opens=java.base/java.lang=ALL-UNNAMED " +
           "--add-opens=java.base/java.util=ALL-UNNAMED")
    
    # Performance Tuning
    .config("spark.sql.shuffle.partitions", "4")
    .config("spark.default.parallelism", "4")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    
    .getOrCreate())

In [18]:
# Test Kafka connectivity before attempting to read
from pyspark.sql.utils import StreamingQueryException

try:
    # Simple test connection
    test_df = (spark.read
        .format("kafka")
        .option("kafka.bootstrap.servers", "kafka-1:9092")
        .option("subscribe", "dummy")
        .option("startingOffsets", "earliest")
        .load())
    print("✅ Kafka test connection successful")
except Exception as e:
    print(f"❌ Kafka connection failed: {e}")
    raise

✅ Kafka test connection successful


In [19]:
## 1. Extract: Read Streaming Data from Kafka

# Define schema for smart meter data
meter_schema = StructType([
    StructField("meter_id", StringType()),
    StructField("timestamp", TimestampType()),
    StructField("kwh_usage", DoubleType()),
    StructField("voltage", IntegerType()),
    StructField("customer_id", StringType()),
    StructField("region", StringType())
])

kafka_df = (spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "kafka-1:9092,kafka-2:9095")
    .option("subscribe", "smart_meter_data")
    .option("startingOffsets", "latest")
    .option("kafka.security.protocol", "PLAINTEXT")
    .option("failOnDataLoss", "false")
    .option("minPartitions", "1")
    .load())

# Parse JSON data
parsed_df = kafka_df.select(
    from_json(col("value").cast("string"), meter_schema).alias("data")
).select("data.*")

# Debug
debug_query = (parsed_df.writeStream
    .format("console")
    .outputMode("append")
    .start())

In [20]:
## 2. Transform: Clean and Enrich Data

# Define validation UDFs
@udf(returnType=BooleanType())
def is_valid_voltage(voltage: int) -> bool:
    """Check if voltage is valid (230V or 240V)."""
    return voltage in [230, 240]

@udf(returnType=BooleanType())
def is_valid_kwh(kwh: float | int) -> bool:
    """Check if kWh usage is within reasonable bounds."""
    return 0 <= kwh <= 20

# Transformation pipeline with proper parentheses
enhanced_df = (parsed_df
    # Current transformations
    .withColumn("processing_time", current_timestamp())
    .withColumn("date", to_date(col("timestamp")))
    .withColumn("hour_of_day", hour(col("timestamp")))
    .withColumn("day_of_week", dayofweek(col("timestamp")))
    .withColumn("cost", 
        when(col("region") == "Auckland", col("kwh_usage") * 0.25)
        .when(col("region") == "Wellington", col("kwh_usage") * 0.23)
        .otherwise(col("kwh_usage") * 0.20))
    .withColumn("is_peak", 
        ((col("hour_of_day") >= 17) & (col("hour_of_day") <= 21)))
    
    # Enhanced data quality checks
    .withColumn("is_weekend", col("day_of_week").isin([1, 7]))
    .withColumn("is_valid_voltage", is_valid_voltage(col("voltage")))
    .withColumn("is_valid_kwh", is_valid_kwh(col("kwh_usage")))
    .withColumn("data_quality_flag",
        when(col("is_valid_voltage") & col("is_valid_kwh"), "VALID")
        .otherwise("INVALID"))
    
    # Improved null handling
    .filter(
        col("meter_id").isNotNull() & 
        col("customer_id").isNotNull() &
        col("timestamp").isNotNull()
    )
    
    # Add record source
    .withColumn("source_system", lit("kafka_stream"))
)

In [25]:
## 3. Load: Write to Postgres

# Function to write batch to PostgreSQL

# Initialize connection pool
postgres_pool = pool.SimpleConnectionPool(
    minconn=1, maxconn=10,
    host="postgres", dbname="postgres",
    user="postgres", password="postgres"
)

@lru_cache(maxsize=1)
def get_existing_customers(cur) -> set[str]:
    """Get all valid customer IDs with caching."""
    cur.execute("SELECT customer_id FROM dim_customer")
    return {row[0] for row in cur.fetchall()}

@lru_cache(maxsize=1)
def get_existing_meters(cur) -> set[str]:
    """Get all valid meter IDs with caching."""
    cur.execute("SELECT meter_id FROM dim_meter")
    return {row[0] for row in cur.fetchall()}

def write_to_postgres(batch_df: DataFrame, batch_id: int) -> None:
    """Write batch data to Postgres with proper validation."""
    if batch_df.isEmpty():
        return

    batch_df.persist()
    conn = None
    
    try:
        pdf = batch_df.select([
            "meter_id", "timestamp", "kwh_usage", "voltage",
            "customer_id", "region", "hour_of_day", "cost",
            "is_peak", "is_weekend", "processing_time",
            "date", "data_quality_flag", "source_system"
        ]).toPandas()

        # Convert timestamps
        pdf['timestamp'] = pd.to_datetime(pdf['timestamp'])
        pdf['processing_time'] = pd.to_datetime(pdf['processing_time'])
        pdf['date'] = pd.to_datetime(pdf['date']).dt.date

        conn = postgres_pool.getconn()
        with conn.cursor() as cur:
            # Get existing references
            existing_customers = get_existing_customers(cur)
            existing_meters = get_existing_meters(cur)
            
            # Handle new meters
            new_meters = set(pdf['meter_id']) - existing_meters
            if new_meters:
                print(f"Batch {batch_id}: Adding {len(new_meters)} new meters")
                execute_batch(cur,
                    """INSERT INTO dim_meter (meter_id, installation_date, is_active)
                       VALUES (%s, CURRENT_DATE, TRUE)
                       ON CONFLICT (meter_id) DO NOTHING""",
                    [(m,) for m in new_meters]
                )
            
            # Filter out non-existent customers
            valid_customers = pdf['customer_id'].isin(existing_customers)
            invalid_customer_count = len(pdf) - sum(valid_customers)
            if invalid_customer_count > 0:
                print(f"Batch {batch_id}: Filtering out {invalid_customer_count} invalid customer records")
                pdf = pdf[valid_customers]
            
            # Main insert with explicit column list
            insert_sql = """
                INSERT INTO fact_smart_meter_readings (
                    meter_id, timestamp, kwh_usage, voltage,
                    customer_id, region, hour_of_day, cost,
                    is_peak, is_weekend, processing_time,
                    date, data_quality_flag, source_system
                ) VALUES (
                    %s, %s, %s, %s, %s, %s, %s, %s,
                    %s, %s, %s, %s, %s, %s
                )
                ON CONFLICT (meter_id, timestamp) 
                DO UPDATE SET
                    kwh_usage = EXCLUDED.kwh_usage,
                    voltage = EXCLUDED.voltage,
                    cost = EXCLUDED.cost,
                    data_quality_flag = EXCLUDED.data_quality_flag,
                    processing_time = EXCLUDED.processing_time
            """
            execute_batch(cur, insert_sql, 
                [tuple(row) for row in pdf.itertuples(index=False)],
                page_size=100
            )
            conn.commit()

    except Exception as e:
        print(f"Error in batch {batch_id}: {str(e)}")
        if conn: conn.rollback()
        raise
    finally:
        batch_df.unpersist()
        if conn: postgres_pool.putconn(conn)

In [26]:
## 4. Load: Write to Minio/S3

# Function to write batch to MinIO
def write_to_minio(batch_df: DataFrame, batch_id: Any) -> None:
    """Write batch data to MinIO using pre-configured Spark session settings."""
    try:
        if batch_df.isEmpty():
            print(f"Skipping empty batch {batch_id}")
            return
       
        # Use consistent path naming with date partitioning
        output_path = f"s3a://default/smart_meter/raw/batch_id={batch_id}/"
        
        try:
            (batch_df.write
                .format("parquet")
                .mode("append")
                .option("compression", "snappy")
                .save(output_path))
            
            print(f"Successfully wrote {batch_df.count()} records to {output_path}")
        except Exception as write_e:
            print(f"Failed to write batch {batch_id}: {str(write_e)}")

    except Exception as e:
        print(f"Unexpected error in batch {batch_id}: {str(e)}")

In [27]:
## 5. Execute the Streaming Pipeline

def run_streaming():
    """Run batch processing ETL pipeline to MinIO and Postgres."""
    try:
        print("Starting streaming queries...")
        
        # Start PostgreSQL writer
        pg_query = (enhanced_df.writeStream
            .foreachBatch(write_to_postgres)
            .option("checkpointLocation", "/tmp/checkpoints/postgres")
            .option("continueOnError", "true")
            .trigger(processingTime="30 seconds")
            .start())
        
        # Start MinIO writer
        minio_query = (enhanced_df.writeStream
            .foreachBatch(write_to_minio)
            .option("checkpointLocation", "/tmp/checkpoints/minio")
            .option("continueOnError", "true")
            .option("maxFilesPerTrigger", "100")
            .option("maxOffsetsPerTrigger", "10000")
            .trigger(processingTime="30 seconds")
            .start())

        print("Streaming queries started successfully")
        print(f"PostgreSQL checkpoint: /tmp/checkpoints/postgres")
        print(f"MinIO checkpoint: /tmp/checkpoints/minio")
        
        # Handle each query separately
        while True:
            pg_status = pg_query.status
            minio_status = minio_query.status
            
            # Status reporting
            status_msg = [
                "\n==== Pipeline Status ====",
                f"PostgreSQL: {pg_status['message']} | Last Progress: {pg_query.lastProgress}",
                f"MinIO: {minio_status['message']} | Last Progress: {minio_query.lastProgress}",
                "========================="
            ]
            print("\n".join(status_msg))
            
            # Exception handling with detailed diagnostics
            if pg_ex := pg_query.exception():
                print(f"\nPOSTGRES ERROR: {str(pg_ex)}")
                print("Last 5 PostgreSQL batches:")
                for p in pg_query.recentProgress[-5:]:
                    print(f"- Batch {p['batchId']}: {p['numInputRows']} rows")
                
            if minio_ex := minio_query.exception():
                print(f"\nMINIO ERROR: {str(minio_ex)}")
                print("Last 5 MinIO batches:")
                for p in minio_query.recentProgress[-5:]:
                    print(f"- Batch {p['batchId']}: {p['numInputRows']} rows")
                
            sleep(10)
            
    except KeyboardInterrupt:
        print("\nUser requested shutdown...")
    except Exception as e:
        print(f"\nCRITICAL ERROR: {str(e)}", file=sys.stderr)
    finally:
        print("\nShutting down streams...")
        for name, q in [("PostgreSQL", pg_query), ("MinIO", minio_query)]:
            if q and q.isActive:
                print(f"Stopping {name} query...")
                try:
                    q.stop()
                except Exception as e:
                    print(f"Error stopping {name} query: {str(e)}")
        print("All streams stopped")
    try:
        with psycopg2.connect(
            host="postgres",
            dbname="postgres",
            user="postgres",
            password="postgres",
            connect_timeout=5
        ) as conn:
            with conn.cursor() as cur:
                # Create dim_customer if not exists
                cur.execute("""
                    CREATE TABLE IF NOT EXISTS dim_customer (
                        customer_id VARCHAR(50) PRIMARY KEY,
                        created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
                    )
                """)
                # Create dim_meter if not exists
                cur.execute("""
                    CREATE TABLE IF NOT EXISTS dim_meter (
                        meter_id VARCHAR(50) PRIMARY KEY,
                        created_at TIMESTAMP WITH TIME ZONE DEFAULT CURRENT_TIMESTAMP
                    )
                """)
                conn.commit()
    except Exception as e:
        print(f"Error initializing tables: {str(e)}")
        raise

In [28]:
## 6. Run ETL
run_streaming()

Starting streaming queries...
Streaming queries started successfully
PostgreSQL checkpoint: /tmp/checkpoints/postgres
MinIO checkpoint: /tmp/checkpoints/minio

==== Pipeline Status ====
PostgreSQL: Processing new data | Last Progress: None
MinIO: Initializing sources | Last Progress: None
Batch 1: Adding 44 new meters
Batch 1: Filtering out 27 invalid customer records
Successfully wrote 364 records to s3a://default/smart_meter/raw/batch_id=4/

==== Pipeline Status ====
PostgreSQL: Processing new data | Last Progress: {'id': '9bfa51e9-f8fd-47ba-afe2-a02f84414361', 'runId': '8c9f2c7c-9fc7-4465-af40-a3ddaf150bf1', 'name': None, 'timestamp': '2025-07-23T20:39:51.352Z', 'batchId': 1, 'numInputRows': 53, 'inputRowsPerSecond': 0.0, 'processedRowsPerSecond': 14.0807651434644, 'durationMs': {'addBatch': 3602, 'commitOffsets': 99, 'getBatch': 0, 'queryPlanning': 53, 'triggerExecution': 3764}, 'stateOperators': [], 'sources': [{'description': 'KafkaV2[Subscribe[smart_meter_data]]', 'startOffset':